# 금융 보고서 분석 
OpenAI 기본 제공 예제 활용 : Financial Analyst Assistant 
https://platform.openai.com/docs/assistants/tools/file-search/quickstart

## File Search 기능을 추가하여 어시스턴트 생성 

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
 
client = OpenAI()
 
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant",
  instructions="You are an expert financial analyst. Use your knowledge base to answer questions about audited financial statements. Confirm the consistency between the document content and your answers. When providing final answers, cite the relevant parts of the documents. Ensure your responses are clear, insightful, and directly address the user's questions. Request additional information or state assumptions when necessary to improve the accuracy of your analysis.",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

In [2]:
assistant

Assistant(id='asst_yyJZUopanB7e8gdBgXZx31Ya', created_at=1720808812, description=None, instructions="You are an expert financial analyst. Use your knowledge base to answer questions about audited financial statements. Confirm the consistency between the document content and your answers. When providing final answers, cite the relevant parts of the documents. Ensure your responses are clear, insightful, and directly address the user's questions. Request additional information or state assumptions when necessary to improve the accuracy of your analysis.", metadata={}, model='gpt-4o', name='Financial Analyst Assistant', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

## 파일을 업로드하고 벡터 스토어에 저장 

In [3]:
vector_store = client.beta.vector_stores.create(name="Financial Statements")
 

file_paths = ["data/NVIDIA 10-K.pdf", "data/NVIDIA 10-Q.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 

print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=2, failed=0, in_progress=0, total=2)


## 벡터 스토어를 사용하도록 어시스턴트 업데이트 

In [4]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

## 스레드 생성 

In [5]:
message_file = client.files.create(
  file=open("data/NVIDIA 10-K.pdf", "rb"), purpose="assistants"
)
 

thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "NVIDIA의 데이터 센터 매출은 AI 및 가속 컴퓨팅 플랫폼에 대한 수요 증가로 인해 2024 회계연도에 상당한 성장을 경험했습니다. 데이터 센터 시장에서 이러한 성장 궤도를 유지하는 데 있어 NVIDIA의 향후 목표와 과제는 무엇입니까?",
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_2PLqUgvryb5ztyv3kXLE8ruI'])


## Run 객체 생성 및 결과 확인 

In [6]:
from typing_extensions import override
from openai import AssistantEventHandler, OpenAI
 
client = OpenAI()
 
class EventHandler(AssistantEventHandler):
    @override
    def on_text_created(self, text) -> None:
        print(f"\nassistant > ", end="", flush=True)

    @override
    def on_tool_call_created(self, tool_call):
        print(f"\nassistant > {tool_call.type}\n", flush=True)

    @override
    def on_message_done(self, message) -> None:
        message_content = message.content[0].text
        annotations = message_content.annotations
        citations = []
        for index, annotation in enumerate(annotations):
            message_content.value = message_content.value.replace(
                annotation.text, f"[{index}]"
            )
            if file_citation := getattr(annotation, "file_citation", None):
                cited_file = client.files.retrieve(file_citation.file_id)
                citations.append(f"[{index}] {cited_file.filename}")

        print(message_content.value)
        print("\n".join(citations))

with client.beta.threads.runs.stream(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Address the user as Hyo Jeong. This is a premium account user. Provide all responses in Korean.",
    event_handler=EventHandler(),
) as stream:
    stream.until_done()


assistant > file_search


assistant > NVIDIA는 2024 회계연도에 데이터 센터 매출이 217% 증가하면서 상당한 성장을 이뤘으며, 이러한 성장을 이어나가기 위해 여러 목표와 과제를 설정하였습니다.

### 목표
1. **AI 및 가속 컴퓨팅 플랫폼 강화**:
   - NVIDIA는 AI 훈련과 추론을 위한 종합적인 가속 컴퓨팅 플랫폼을 제공합니다 .
   - NVIDIA DGX Cloud와 같은 AI 트레이닝 서비스 플랫폼을 도입하고, NeMo를 통해 기업용 LLMs(대형 언어 모델) 구축 솔루션을 제공합니다 .
   - 새로운 Hopper GPU 아키텍처와 함께 데이터 센터 제품의 출시 속도를 높여 AI 기회를 확대할 계획입니다  .

2. **고객 수요 및 공급 관리**:
   - NVIDIA는 2025 회계연도 동안 고객 수요 표시를 수집하고, 새로운 데이터 센터 제품 출시 시기를 조정하여 수요에 맞춰 공급을 관리할 계획입니다 .

3. **기술 생태계 확장**:
   - GPU, CPU, DPU 모든 주요 처리 장치를 포함하는 AI 서버를 통한 컴퓨팅 솔루션 제공.
   - NVIDIA AI Enterprise 소프트웨어를 통한 기업용 AI 응용 프로그램 지원 .

### 과제
1. **제품 전환 및 공급망 관리**:
   - 새로운 제품과 이전 아키텍처 제품을 동시에 출하하면서 수요와 공급을 적절히 관리하는 것이 복잡합니다.
   - 새로운 공급업체 통합, 준비된 제조 및 용량 계약으로 인한 복잡성 증가와 실행 위험이 존재합니다  .

2. **수요 예측과 재고 관리**:
   - 제품 개발 주기 단축, 새로운 비즈니스 라인 진출 등의 이유로 장기 수요 예측의 어려움이 존재합니다  .
   - AI 및 HPC(고성능 컴퓨팅) 작업의 확대로 데이터 센터 전체의 컴퓨팅으로의 전환이 필요합니다 .

3. **기술 개발과 품질 문제**:
   - 높은 제품 전환 빈도와 새로운 기술 도입으로 인한 품질 문제 및 생산 지연 